In [34]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.train.automl.run import AutoMLRun as run
from azureml.core.model import Model

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.dnn import PyTorch

In [8]:
ws = Workspace.from_config(path="./config.json")

In [20]:
!pwd
!ls

/mnt/batch/tasks/shared/LS_root/mounts/clusters/cheapcheap/code/users/aryanmisra
condadep.yml	     Descriptors.ipynb	     stable-req.txt
config.json	     docker-env.ipynb	     synbio-train
copy2_trainer.ipynb  pytenv		     test1
copy_trainer.ipynb   pytorchestimator.ipynb  test101
data		     rdkit_testing_ground    trainer.ipynb
deploy.ipynb	     score11.ipynb	     train-main


In [9]:
project_folder = './train-main'
experiment_name = 'synbio7'
experiment = Experiment(ws, name=experiment_name)
cluster_name = "gpu-compute"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print(f'Found existing compute target: {compute_target}')

Found existing compute target: AmlCompute(workspace=Workspace.create(name='ml-main', subscription_id='f1be6293-6d1c-4e77-bb06-b2083e9aeb20', resource_group='synbiolic-main'), name=gpu-compute, id=/subscriptions/f1be6293-6d1c-4e77-bb06-b2083e9aeb20/resourceGroups/synbiolic-main/providers/Microsoft.MachineLearningServices/workspaces/ml-main/computes/gpu-compute, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags=None)


In [10]:
pytenv = Environment(name="pytenv")
conda_dep = CondaDependencies().create(python_version='3.6')
# conda_dep.add_pip_package("olefile==0.4.6") #add this first
conda_dep.add_channel('deepchem')
conda_dep.add_channel('omnia')
conda_dep.add_channel('conda-forge')
conda_dep.add_channel('rdkit')
# conda_dep.add_channel('pytorch')
conda_dep.add_conda_package("deepchem-gpu=2.1.0")
conda_dep.add_pip_package("azureml-defaults")
# conda_dep.add_conda_package("astor")
conda_dep.add_conda_package("torchvision")
conda_dep.add_conda_package("pytorch")
# conda_dep.add_conda_package("cudatoolkit=10.1")
# conda_dep.add_pip_package("astor==0.8.1")
# conda_dep.add_conda_package("olefile==0.46")
conda_dep.save(path="./train-main/condadep.yml")
pytenv.python.conda_dependencies=conda_dep

In [11]:
estimator = PyTorch(source_directory=project_folder, 
                    compute_target=compute_target,
                    entry_script='train.py',
                    use_gpu=True,
                    conda_dependencies_file="./condadep.yml")

WARNING - If environment_definition or conda_dependencies_file is specified, Azure ML will not install any framework related packages on behalf of the user.
WARNING - framework_version is not specified, defaulting to version 1.3.


In [ ]:
# model = run.register_model(description = "model_deploy", 
                          # tags={"area": "synbiolic"},
                          # model_path="./train-main/outputs/model_deploy.pkl",
                          # model_name="model_deploy_synbiolic")
model = Model.register(workspace=ws,
                      model_path="./train-main/outputs/",
                      model_name="model_deploy_synbiolic", 
                      description="runn")

rrun = experiment.submit(estimator)
rrun.wait_for_completion(show_output=True)

Registering model model_deploy_synbiolic
RunId: synbio7_1584503691_07e403e8
Web View: https://ml.azure.com/experiments/synbio7/runs/synbio7_1584503691_07e403e8?wsid=/subscriptions/f1be6293-6d1c-4e77-bb06-b2083e9aeb20/resourcegroups/synbiolic-main/workspaces/ml-main

Streaming azureml-logs/20_image_build_log.txt

2020/03/18 03:55:22 Downloading source code...
2020/03/18 03:55:23 Finished downloading source code
2020/03/18 03:55:24 Creating Docker network: acb_default_network, driver: 'bridge'
2020/03/18 03:55:24 Successfully set up Docker network: acb_default_network
2020/03/18 03:55:24 Setting up Docker configuration...
2020/03/18 03:55:25 Successfully set up Docker configuration
2020/03/18 03:55:25 Logging in to registry: mlmain2c9169d7.azurecr.io
2020/03/18 03:55:26 Successfully logged into mlmain2c9169d7.azurecr.io
2020/03/18 03:55:26 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/03/18 03:55:26 Scanning for dependencies